In [1]:
import os
import numpy as np
import pandas as pd
from datetime import date

from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve
from sklearn.preprocessing import MinMaxScaler

DATA_ROOT = "./ml100marathon-02-01/"

In [2]:
dfoff = pd.read_csv(os.path.join(DATA_ROOT,'train_offline.csv'))
dftest = pd.read_csv(os.path.join(DATA_ROOT,'test_offline.csv'))
dftest = dftest[~dftest.Coupon_id.isna()]
dftest.reset_index(drop=True, inplace=True)
print(dfoff.shape)
print(dftest.shape)
print(dfoff.head(10))
print(dftest.head(10))

(1160742, 7)
(306313, 6)
   User_id  Merchant_id  Coupon_id Discount_rate  Distance  Date_received  \
0  1439408         2632        NaN           NaN       0.0            NaN   
1  1439408         2632     8591.0          20:1       0.0     20160217.0   
2  1439408         2632     1078.0          20:1       0.0     20160319.0   
3  1832624         3381     7610.0        200:20       0.0     20160429.0   
4  2029232         3381    11951.0        200:20       1.0     20160129.0   
5  2223968         3381     9776.0          10:5       2.0     20160129.0   
6    73611         2099    12034.0        100:10       NaN     20160207.0   
7   163606         1569     5054.0        200:30      10.0     20160421.0   
8  3273056         4833     7802.0        200:20      10.0     20160130.0   
9    94107         3381     7610.0        200:20       2.0     20160412.0   

         Date  
0  20160217.0  
1         NaN  
2         NaN  
3         NaN  
4         NaN  
5         NaN  
6         NaN  

In [3]:
## Creat target label 
"""
According to the definition, 
1) buy with coupon within (include) 15 days ==> 1
2) buy with coupon but out of 15 days ==> 0
3) buy without coupon ==> -1 (we don't care)
"""
def label(row):
    if np.isnan(row['Date_received']):
        return -1
    if not np.isnan(row['Date']):
        td = pd.to_datetime(row['Date'], format='%Y%m%d') -  pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0

dfoff["label"] = dfoff.apply(label, axis=1)
dfoff["label"].value_counts()

 0    710665
-1    413773
 1     36304
Name: label, dtype: int64

In [4]:
# Generate features - weekday acquired coupon
def getWeekday(row):
    if (np.isnan(row)) or (row==-1):
        return row
    else:
        return pd.to_datetime(row, format = "%Y%m%d").dayofweek+1 # add one to make it from 0~6 -> 1~7

dfoff['weekday'] = dfoff['Date_received'].apply(getWeekday)
dftest['weekday'] = dftest['Date_received'].apply(getWeekday)

# weekday_type (weekend = 1)
dfoff['weekday_type'] = dfoff['weekday'].astype('str').apply(lambda x : 1 if x in [6,7] else 0 ) # apply to trainset
dftest['weekday_type'] = dftest['weekday'].astype('str').apply(lambda x : 1 if x in [6,7] else 0 ) # apply to testset

In [5]:
print(dfoff['weekday'][:10])
print(dfoff['weekday_type'][:10])

0    NaN
1    3.0
2    6.0
3    5.0
4    5.0
5    5.0
6    7.0
7    4.0
8    6.0
9    2.0
Name: weekday, dtype: float64
0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: weekday_type, dtype: int64


In [6]:
weekdaycols = ['weekday_' + str(i) for i in range(1,8)]
print(weekdaycols)

tmpdf = pd.get_dummies(dfoff['weekday'].replace(-1, np.nan))
tmpdf.columns = weekdaycols
dfoff[weekdaycols] = tmpdf

tmpdf = pd.get_dummies(dftest['weekday'].replace(-1, np.nan))
tmpdf.columns = weekdaycols
dftest[weekdaycols] = tmpdf

['weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


In [7]:
print(dfoff[weekdaycols][:10])

   weekday_1  weekday_2  weekday_3  weekday_4  weekday_5  weekday_6  weekday_7
0          0          0          0          0          0          0          0
1          0          0          1          0          0          0          0
2          0          0          0          0          0          1          0
3          0          0          0          0          1          0          0
4          0          0          0          0          1          0          0
5          0          0          0          0          1          0          0
6          0          0          0          0          0          0          1
7          0          0          0          1          0          0          0
8          0          0          0          0          0          1          0
9          0          1          0          0          0          0          0


In [8]:
# Generate features - coupon discount and distance
def getDiscountType(row):
    if row == 'null':
        return 'null'
    elif ':' in row:
        return 1
    else:
        return 0

def convertRate(row):
    """Convert discount to rate"""
    if row == 'null':
        return 1.0
    elif ':' in row:
        rows = row.split(':')
        return 1.0 - float(rows[1])/float(rows[0])
    else:
        return float(row)

def getDiscountMan(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[0])
    else:
        return 0

def getDiscountJian(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[1])
    else:
        return 0

def processData(df):
    
    # convert discunt_rate
    df['discount_rate'] = df['Discount_rate'].astype('str').apply(convertRate)
    df['discount_man'] = df['Discount_rate'].astype('str').apply(getDiscountMan)
    df['discount_jian'] = df['Discount_rate'].astype('str').apply(getDiscountJian)
    df['discount_type'] = df['Discount_rate'].astype('str').apply(getDiscountType)
    
    # convert distance
    df.loc[df.Distance.isna(), "Distance"] = 99
    return df

dfoff = processData(dfoff)
dftest = processData(dftest)

In [9]:
print(dfoff.shape)
print(dfoff.head())
print(dftest.shape)
print(dftest.head())

(1160742, 21)
   User_id  Merchant_id  Coupon_id Discount_rate  Distance  Date_received  \
0  1439408         2632        NaN           NaN       0.0            NaN   
1  1439408         2632     8591.0          20:1       0.0     20160217.0   
2  1439408         2632     1078.0          20:1       0.0     20160319.0   
3  1832624         3381     7610.0        200:20       0.0     20160429.0   
4  2029232         3381    11951.0        200:20       1.0     20160129.0   

         Date  label  weekday  weekday_type  ...  weekday_2  weekday_3  \
0  20160217.0     -1      NaN             0  ...          0          0   
1         NaN      0      3.0             0  ...          0          1   
2         NaN      0      6.0             0  ...          0          0   
3         NaN      0      5.0             0  ...          0          0   
4         NaN      0      5.0             0  ...          0          0   

   weekday_4  weekday_5  weekday_6  weekday_7  discount_rate  discount_man  \


In [10]:
## Naive model
## date < date_cut = train data else valid data
def split_train_valid(row, date_cut="20160416"):
    is_train = True if pd.to_datetime(row, format="%Y%m%d") < pd.to_datetime(date_cut, format="%Y%m%d") else False
    return is_train
    
df = dfoff[dfoff['label'] != -1].copy()
df["is_train"] = df["Date_received"].apply(split_train_valid)
train = df[df["is_train"]]
valid = df[~df["is_train"]]
train.reset_index(drop=True, inplace=True)
valid.reset_index(drop=True, inplace=True)
print("Train size: {}, #positive: {}".format(len(train), train["label"].sum()))
print("Valid size: {}, #positive: {}".format(len(valid), valid["label"].sum()))

Train size: 667753, #positive: 32472
Valid size: 79216, #positive: 3832


In [11]:
print(train.head())
print(valid.head())

   User_id  Merchant_id  Coupon_id Discount_rate  Distance  Date_received  \
0  1439408         2632     8591.0          20:1       0.0     20160217.0   
1  1439408         2632     1078.0          20:1       0.0     20160319.0   
2  2029232         3381    11951.0        200:20       1.0     20160129.0   
3  2223968         3381     9776.0          10:5       2.0     20160129.0   
4    73611         2099    12034.0        100:10      99.0     20160207.0   

   Date  label  weekday  weekday_type  ...  weekday_3  weekday_4  weekday_5  \
0   NaN      0      3.0             0  ...          1          0          0   
1   NaN      0      6.0             0  ...          0          0          0   
2   NaN      0      5.0             0  ...          0          0          1   
3   NaN      0      5.0             0  ...          0          0          1   
4   NaN      0      7.0             0  ...          0          0          0   

   weekday_6  weekday_7  discount_rate  discount_man  discount

In [12]:
train.to_csv(os.path.join(DATA_ROOT,"train.csv"), index=False) 
valid.to_csv(os.path.join(DATA_ROOT,"valid.csv"), index=False) 
dftest.to_csv(os.path.join(DATA_ROOT,"test.csv"), index=False) 
print('Done!')

Done!
